In [1]:
import pandas as pd
from tqdm import tqdm
from itertools import combinations

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.preprocessing import TargetEncoder

import warnings
warnings.filterwarnings("ignore")

import lightgbm as lgb

In [2]:
df_subm = pd.read_csv('sample_submission.csv', index_col='id')
y_pred = np.zeros(len(df_subm))

In [3]:
def train(X_train,y_train,X_test,y_pred):
    model = lgb.LGBMRegressor(
    n_iter=50000,
    max_depth=-1,
    num_leaves=1024,
    colsample_bytree=0.2,
    subsample = 0.7,
    learning_rate=0.005,
    objective='l2',
    metric='rmse', 
    verbosity=-1,
    max_bin=1024,
    random_state = 42
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_train,y_train),(X_valid, y_valid)],
        callbacks=[lgb.log_evaluation(100), lgb.early_stopping(200)],
    )

    y_pred_valid = model.predict(X_valid)
    y_oof = model.predict(X_test)
    
    y_pred += y_oof
    return y_oof, y_pred_valid,model

In [ ]:
#training's rmse: 4.01842	valid_1's rmse: 11.8759
##training's rmse: 5.46618	valid_1's rmse: 11.8854
X_train = pd.read_parquet('X_train_1.parquet.gzip') 
X_valid = pd.read_parquet('X_valid_1.parquet.gzip')
y_train = pd.read_csv('y_train_1.csv') 
y_valid = pd.read_csv('y_valid_1.csv')
X_test = pd.read_parquet('X_test_1.parquet.gzip')

y_train = y_train['Listening_Time_minutes']
y_valid = y_valid['Listening_Time_minutes']

X_train['Genre'] = X_train['Genre'].astype('category')
X_train['Podcast_Name'] = X_train['Podcast_Name'].astype('category')
X_train['Publication_Day'] = X_train['Publication_Day'].astype('category')
X_train['Publication_Time'] = X_train['Publication_Time'].astype('category')
X_train['Episode_Sentiment'] = X_train['Episode_Sentiment'].astype('category')
X_train['Episode_Title'] = X_train['Episode_Title'].astype('category')

X_valid['Genre'] = X_valid['Genre'].astype('category')
X_valid['Podcast_Name'] = X_valid['Podcast_Name'].astype('category')
X_valid['Publication_Day'] = X_valid['Publication_Day'].astype('category')
X_valid['Publication_Time'] = X_valid['Publication_Time'].astype('category')
X_valid['Episode_Sentiment'] = X_valid['Episode_Sentiment'].astype('category')
X_valid['Episode_Title'] = X_valid['Episode_Title'].astype('category')

X_test['Genre'] = X_test['Genre'].astype('category')
X_test['Podcast_Name'] = X_test['Podcast_Name'].astype('category')
X_test['Publication_Day'] = X_test['Publication_Day'].astype('category')
X_test['Publication_Time'] = X_test['Publication_Time'].astype('category')
X_test['Episode_Sentiment'] = X_test['Episode_Sentiment'].astype('category')
X_test['Episode_Title'] = X_test['Episode_Title'].astype('category')
print('1')

for x in [5,10,15,20,25,50,100,150]:
    name = 'Episode_Length_minutes' + str(x)
    X_train["Episode_Length_minutes_copy"] = X_train["Episode_Length_minutes"].copy()
    X_valid["Episode_Length_minutes_copy"] = X_valid["Episode_Length_minutes"].copy()
    X_test["Episode_Length_minutes_copy"] = X_test["Episode_Length_minutes"].copy()
    
    ser, bins = pd.qcut(X_train["Episode_Length_minutes_copy"], x, retbins=True, labels=False)
    X_train[name] = pd.cut(X_train['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
    X_valid[name] = pd.cut(X_valid['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
    X_test[name] = pd.cut(X_test['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
print(X_valid.shape,len(y_valid))

for x in ['mean']:
    for w in ['Listening_Time_minutes']:
        for z in ['Episode_Length_minutes5','Episode_Length_minutes10','Episode_Length_minutes15','Episode_Length_minutes20'
                 ,'Episode_Length_minutes25','Episode_Length_minutes50','Episode_Length_minutes100','Episode_Length_minutes150']:
            name = w + x + z
            a = X_train.groupby(z)[w].apply(x).reset_index(name=name)
            X_train = X_train.merge(a,on=z,how='left')
            X_test = X_test.merge(a,on=z,how='left')      
            X_valid = X_valid.merge(a,on=z,how='left') 
print(X_valid.shape,len(y_valid))

QUANTILE_LIST = []
for x in [0.05,0.10,0.40,0.45,0.55,0.60,0.90,0.95]:
    for w in ['Listening_Time_minutes']:
        for z in ['Publication_Day','Podcast_Name','Publication_Time','Episode_Title','Genre','NaNs'
                  , 'Episode_Length_minutes_round0', 'Episode_Length_minutes_digit0','Episode_Length_minutes_digit1']:
            name = w + str(x) + z
            a = X_train.groupby(z)[w].quantile(x).reset_index(name=name)
            X_train = X_train.merge(a,on=z,how='left')
            X_test = X_test.merge(a,on=z,how='left')      
            X_valid = X_valid.merge(a,on=z,how='left')
            QUANTILE_LIST.append(name)
print(X_valid.shape,len(y_valid))

X_train = X_train.drop(columns=['Listening_Time_minutes'])
X_valid = X_valid.drop(columns=['Listening_Time_minutes'])

y_oof, y_pred_valid,model_1 = train(X_train,y_train,X_test,y_pred)
np.savetxt("y_oof_1.csv", y_oof, delimiter=",",header='Listening_Time_minutes')
np.savetxt("y_pred_valid_1.csv", y_pred_valid, delimiter=",",header='Listening_Time_minutes')

1
(158974, 872) 158974
(158974, 880) 158974
(158974, 952) 158974
Training until validation scores don't improve for 200 rounds
[100]	training's rmse: 18.9847	valid_1's rmse: 19.211
[200]	training's rmse: 14.7023	valid_1's rmse: 15.1814
[300]	training's rmse: 12.5691	valid_1's rmse: 13.3437
[400]	training's rmse: 11.4997	valid_1's rmse: 12.558
[500]	training's rmse: 10.9139	valid_1's rmse: 12.2269
[600]	training's rmse: 10.5417	valid_1's rmse: 12.0823
[700]	training's rmse: 10.2655	valid_1's rmse: 12.0127
[800]	training's rmse: 10.0372	valid_1's rmse: 11.977
[900]	training's rmse: 9.8353	valid_1's rmse: 11.9558


In [ ]:
fig, ax = plt.subplots(figsize=(10, 20))
lgb.plot_importance(model_1,max_num_features=50,importance_type='gain',ax=ax)

In [ ]:
#training's rmse: 4.61699	valid_1's rmse: 11.8012
X_train = pd.read_parquet('X_train_2.parquet.gzip') 
X_valid = pd.read_parquet('X_valid_2.parquet.gzip')
y_train = pd.read_csv('y_train_2.csv') 
y_valid = pd.read_csv('y_valid_2.csv')
X_test = pd.read_parquet('X_test_2.parquet.gzip')

y_train = y_train['Listening_Time_minutes']
y_valid = y_valid['Listening_Time_minutes']

X_train['Genre'] = X_train['Genre'].astype('category')
X_train['Podcast_Name'] = X_train['Podcast_Name'].astype('category')
X_train['Publication_Day'] = X_train['Publication_Day'].astype('category')
X_train['Publication_Time'] = X_train['Publication_Time'].astype('category')
X_train['Episode_Sentiment'] = X_train['Episode_Sentiment'].astype('category')

X_valid['Genre'] = X_valid['Genre'].astype('category')
X_valid['Podcast_Name'] = X_valid['Podcast_Name'].astype('category')
X_valid['Publication_Day'] = X_valid['Publication_Day'].astype('category')
X_valid['Publication_Time'] = X_valid['Publication_Time'].astype('category')
X_valid['Episode_Sentiment'] = X_valid['Episode_Sentiment'].astype('category')

X_test['Genre'] = X_test['Genre'].astype('category')
X_test['Podcast_Name'] = X_test['Podcast_Name'].astype('category')
X_test['Publication_Day'] = X_test['Publication_Day'].astype('category')
X_test['Publication_Time'] = X_test['Publication_Time'].astype('category')
X_test['Episode_Sentiment'] = X_test['Episode_Sentiment'].astype('category')

for x in X_train.columns[191:-5]:
    a = x + 'div' + 'Episode_Length_minutes'
    X_train[a] = X_train['Episode_Length_minutes'] / X_train[x]
    X_valid[a] = X_valid['Episode_Length_minutes'] / X_valid[x]
    X_test[a] = X_test['Episode_Length_minutes'] / X_test[x]

for x in [5,10,15,20,25,50,100]:
    name = 'Episode_Length_minutes' + str(x)
    X_train["Episode_Length_minutes_copy"] = X_train["Episode_Length_minutes"].copy()
    X_valid["Episode_Length_minutes_copy"] = X_valid["Episode_Length_minutes"].copy()
    X_test["Episode_Length_minutes_copy"] = X_test["Episode_Length_minutes"].copy()
    
    ser, bins = pd.qcut(X_train["Episode_Length_minutes_copy"], x, retbins=True, labels=False)
    X_train[name] = pd.cut(X_train['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
    X_valid[name] = pd.cut(X_valid['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
    X_test[name] = pd.cut(X_test['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')

for x in ['mean','quantile']:
    for w in ['Listening_Time_minutes']:
        for z in ['Episode_Length_minutes5','Episode_Length_minutes10','Episode_Length_minutes15','Episode_Length_minutes20'
                 ,'Episode_Length_minutes25','Episode_Length_minutes50','Episode_Length_minutes100']:
            name = w + x + z
            a = X_train.groupby(z)[w].apply(x).reset_index(name=name)
            X_train = X_train.merge(a,on=z,how='left')
            X_test = X_test.merge(a,on=z,how='left')      
            X_valid = X_valid.merge(a,on=z,how='left') 
print(X_valid.shape,len(y_valid))
print('8')

X_train['SinEpLen'] = np.sin(2*np.pi * X_train['Episode_Length_minutes']/60 )
X_train['CosEpLen'] = np.cos(2*np.pi * X_train['Episode_Length_minutes']/60 )
X_train["ELen_Int"] = np.floor(X_train["Episode_Length_minutes"])
X_train["ELen_Dec"] = X_train["Episode_Length_minutes"] - X_train["ELen_Int"]
X_train['Long_Episode'] = (X_train['Episode_Length_minutes'] > 75).astype(int)

X_test['SinEpLen'] = np.sin(2*np.pi * X_test['Episode_Length_minutes']/60 )
X_test['CosEpLen'] = np.cos(2*np.pi * X_test['Episode_Length_minutes']/60 )
X_test["ELen_Int"] = np.floor(X_test["Episode_Length_minutes"])
X_test["ELen_Dec"] = X_test["Episode_Length_minutes"] - X_test["ELen_Int"]
X_test['Long_Episode'] = (X_test['Episode_Length_minutes'] > 75).astype(int)

X_valid['SinEpLen'] = np.sin(2*np.pi * X_valid['Episode_Length_minutes']/60 )
X_valid['CosEpLen'] = np.cos(2*np.pi * X_valid['Episode_Length_minutes']/60 )
X_valid["ELen_Int"] = np.floor(X_valid["Episode_Length_minutes"])
X_valid["ELen_Dec"] = X_valid["Episode_Length_minutes"] - X_valid["ELen_Int"]
X_valid['Long_Episode'] = (X_valid['Episode_Length_minutes'] > 75).astype(int)

X_train = X_train.drop(columns=['Listening_Time_minutes'])
X_valid = X_valid.drop(columns=['Listening_Time_minutes'])

y_oof, y_pred_valid,model_2 = train(X_train,y_train,X_test,y_pred)
np.savetxt("y_oof_2.csv", y_oof, delimiter=",",header='Listening_Time_minutes')
np.savetxt("y_pred_valid_2.csv", y_pred_valid, delimiter=",",header='Listening_Time_minutes')
print('Done')

In [ ]:
#training's rmse: 5.64439	valid_1's rmse: 11.8389
X_train = pd.read_parquet('X_train_3.parquet.gzip') 
X_valid = pd.read_parquet('X_valid_3.parquet.gzip')
y_train = pd.read_csv('y_train_3.csv') 
y_valid = pd.read_csv('y_valid_3.csv')
X_test = pd.read_parquet('X_test_3.parquet.gzip')

y_train = y_train['Listening_Time_minutes']
y_valid = y_valid['Listening_Time_minutes']

X_train['Genre'] = X_train['Genre'].astype('category')
X_train['Podcast_Name'] = X_train['Podcast_Name'].astype('category')
X_train['Publication_Day'] = X_train['Publication_Day'].astype('category')
X_train['Publication_Time'] = X_train['Publication_Time'].astype('category')
X_train['Episode_Sentiment'] = X_train['Episode_Sentiment'].astype('category')

X_valid['Genre'] = X_valid['Genre'].astype('category')
X_valid['Podcast_Name'] = X_valid['Podcast_Name'].astype('category')
X_valid['Publication_Day'] = X_valid['Publication_Day'].astype('category')
X_valid['Publication_Time'] = X_valid['Publication_Time'].astype('category')
X_valid['Episode_Sentiment'] = X_valid['Episode_Sentiment'].astype('category')

X_test['Genre'] = X_test['Genre'].astype('category')
X_test['Podcast_Name'] = X_test['Podcast_Name'].astype('category')
X_test['Publication_Day'] = X_test['Publication_Day'].astype('category')
X_test['Publication_Time'] = X_test['Publication_Time'].astype('category')
X_test['Episode_Sentiment'] = X_test['Episode_Sentiment'].astype('category')

for x in X_train.columns[191:-5]:
    a = x + 'div' + 'Episode_Length_minutes'
    X_train[a] = X_train['Episode_Length_minutes'] / X_train[x]
    X_valid[a] = X_valid['Episode_Length_minutes'] / X_valid[x]
    X_test[a] = X_test['Episode_Length_minutes'] / X_test[x]

for x in [5,10,15,20,25,50,100]:
    name = 'Episode_Length_minutes' + str(x)
    X_train["Episode_Length_minutes_copy"] = X_train["Episode_Length_minutes"].copy()
    X_valid["Episode_Length_minutes_copy"] = X_valid["Episode_Length_minutes"].copy()
    X_test["Episode_Length_minutes_copy"] = X_test["Episode_Length_minutes"].copy()
    
    ser, bins = pd.qcut(X_train["Episode_Length_minutes_copy"], x, retbins=True, labels=False)
    X_train[name] = pd.cut(X_train['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
    X_valid[name] = pd.cut(X_valid['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
    X_test[name] = pd.cut(X_test['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')

for x in ['mean','quantile']:
    for w in ['Listening_Time_minutes']:
        for z in ['Episode_Length_minutes5','Episode_Length_minutes10','Episode_Length_minutes15','Episode_Length_minutes20'
                 ,'Episode_Length_minutes25','Episode_Length_minutes50','Episode_Length_minutes100']:
            name = w + x + z
            a = X_train.groupby(z)[w].apply(x).reset_index(name=name)
            X_train = X_train.merge(a,on=z,how='left')
            X_test = X_test.merge(a,on=z,how='left')      
            X_valid = X_valid.merge(a,on=z,how='left') 
print(X_valid.shape,len(y_valid))
print('8')

X_train['SinEpLen'] = np.sin(2*np.pi * X_train['Episode_Length_minutes']/60 )
X_train['CosEpLen'] = np.cos(2*np.pi * X_train['Episode_Length_minutes']/60 )
X_train["ELen_Int"] = np.floor(X_train["Episode_Length_minutes"])
X_train["ELen_Dec"] = X_train["Episode_Length_minutes"] - X_train["ELen_Int"]
X_train['Long_Episode'] = (X_train['Episode_Length_minutes'] > 75).astype(int)

X_test['SinEpLen'] = np.sin(2*np.pi * X_test['Episode_Length_minutes']/60 )
X_test['CosEpLen'] = np.cos(2*np.pi * X_test['Episode_Length_minutes']/60 )
X_test["ELen_Int"] = np.floor(X_test["Episode_Length_minutes"])
X_test["ELen_Dec"] = X_test["Episode_Length_minutes"] - X_test["ELen_Int"]
X_test['Long_Episode'] = (X_test['Episode_Length_minutes'] > 75).astype(int)

X_valid['SinEpLen'] = np.sin(2*np.pi * X_valid['Episode_Length_minutes']/60 )
X_valid['CosEpLen'] = np.cos(2*np.pi * X_valid['Episode_Length_minutes']/60 )
X_valid["ELen_Int"] = np.floor(X_valid["Episode_Length_minutes"])
X_valid["ELen_Dec"] = X_valid["Episode_Length_minutes"] - X_valid["ELen_Int"]
X_valid['Long_Episode'] = (X_valid['Episode_Length_minutes'] > 75).astype(int)

X_train = X_train.drop(columns=['Listening_Time_minutes'])
X_valid = X_valid.drop(columns=['Listening_Time_minutes'])

y_oof, y_pred_valid,model_3 = train(X_train,y_train,X_test,y_pred)
np.savetxt("y_oof_3.csv", y_oof, delimiter=",",header='Listening_Time_minutes')
np.savetxt("y_pred_valid_3.csv", y_pred_valid, delimiter=",",header='Listening_Time_minutes')
print('Done')

In [ ]:
lgb.plot_importance(model_3,max_num_features=20)

In [ ]:
#training's rmse: 4.70523	valid_1's rmse: 11.8398
X_train = pd.read_parquet('X_train_4.parquet.gzip') 
X_valid = pd.read_parquet('X_valid_4.parquet.gzip')
y_train = pd.read_csv('y_train_4.csv') 
y_valid = pd.read_csv('y_valid_4.csv')
X_test = pd.read_parquet('X_test_4.parquet.gzip')

y_train = y_train['Listening_Time_minutes']
y_valid = y_valid['Listening_Time_minutes']

X_train['Genre'] = X_train['Genre'].astype('category')
X_train['Podcast_Name'] = X_train['Podcast_Name'].astype('category')
X_train['Publication_Day'] = X_train['Publication_Day'].astype('category')
X_train['Publication_Time'] = X_train['Publication_Time'].astype('category')
X_train['Episode_Sentiment'] = X_train['Episode_Sentiment'].astype('category')

X_valid['Genre'] = X_valid['Genre'].astype('category')
X_valid['Podcast_Name'] = X_valid['Podcast_Name'].astype('category')
X_valid['Publication_Day'] = X_valid['Publication_Day'].astype('category')
X_valid['Publication_Time'] = X_valid['Publication_Time'].astype('category')
X_valid['Episode_Sentiment'] = X_valid['Episode_Sentiment'].astype('category')

X_test['Genre'] = X_test['Genre'].astype('category')
X_test['Podcast_Name'] = X_test['Podcast_Name'].astype('category')
X_test['Publication_Day'] = X_test['Publication_Day'].astype('category')
X_test['Publication_Time'] = X_test['Publication_Time'].astype('category')
X_test['Episode_Sentiment'] = X_test['Episode_Sentiment'].astype('category')

for x in X_train.columns[191:-5]:
    a = x + 'div' + 'Episode_Length_minutes'
    X_train[a] = X_train['Episode_Length_minutes'] / X_train[x]
    X_valid[a] = X_valid['Episode_Length_minutes'] / X_valid[x]
    X_test[a] = X_test['Episode_Length_minutes'] / X_test[x]

for x in [5,10,15,20,25,50,100]:
    name = 'Episode_Length_minutes' + str(x)
    X_train["Episode_Length_minutes_copy"] = X_train["Episode_Length_minutes"].copy()
    X_valid["Episode_Length_minutes_copy"] = X_valid["Episode_Length_minutes"].copy()
    X_test["Episode_Length_minutes_copy"] = X_test["Episode_Length_minutes"].copy()
    
    ser, bins = pd.qcut(X_train["Episode_Length_minutes_copy"], x, retbins=True, labels=False)
    X_train[name] = pd.cut(X_train['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
    X_valid[name] = pd.cut(X_valid['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
    X_test[name] = pd.cut(X_test['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')

for x in ['mean','quantile']:
    for w in ['Listening_Time_minutes']:
        for z in ['Episode_Length_minutes5','Episode_Length_minutes10','Episode_Length_minutes15','Episode_Length_minutes20'
                 ,'Episode_Length_minutes25','Episode_Length_minutes50','Episode_Length_minutes100']:
            name = w + x + z
            a = X_train.groupby(z)[w].apply(x).reset_index(name=name)
            X_train = X_train.merge(a,on=z,how='left')
            X_test = X_test.merge(a,on=z,how='left')      
            X_valid = X_valid.merge(a,on=z,how='left') 
print(X_valid.shape,len(y_valid))
print('8')

X_train['SinEpLen'] = np.sin(2*np.pi * X_train['Episode_Length_minutes']/60 )
X_train['CosEpLen'] = np.cos(2*np.pi * X_train['Episode_Length_minutes']/60 )
X_train["ELen_Int"] = np.floor(X_train["Episode_Length_minutes"])
X_train["ELen_Dec"] = X_train["Episode_Length_minutes"] - X_train["ELen_Int"]
X_train['Long_Episode'] = (X_train['Episode_Length_minutes'] > 75).astype(int)

X_test['SinEpLen'] = np.sin(2*np.pi * X_test['Episode_Length_minutes']/60 )
X_test['CosEpLen'] = np.cos(2*np.pi * X_test['Episode_Length_minutes']/60 )
X_test["ELen_Int"] = np.floor(X_test["Episode_Length_minutes"])
X_test["ELen_Dec"] = X_test["Episode_Length_minutes"] - X_test["ELen_Int"]
X_test['Long_Episode'] = (X_test['Episode_Length_minutes'] > 75).astype(int)

X_valid['SinEpLen'] = np.sin(2*np.pi * X_valid['Episode_Length_minutes']/60 )
X_valid['CosEpLen'] = np.cos(2*np.pi * X_valid['Episode_Length_minutes']/60 )
X_valid["ELen_Int"] = np.floor(X_valid["Episode_Length_minutes"])
X_valid["ELen_Dec"] = X_valid["Episode_Length_minutes"] - X_valid["ELen_Int"]
X_valid['Long_Episode'] = (X_valid['Episode_Length_minutes'] > 75).astype(int)

X_train = X_train.drop(columns=['Listening_Time_minutes'])
X_valid = X_valid.drop(columns=['Listening_Time_minutes'])

y_oof, y_pred_valid,model_4 = train(X_train,y_train,X_test,y_pred)
np.savetxt("y_oof_4.csv", y_oof, delimiter=",",header='Listening_Time_minutes')
np.savetxt("y_pred_valid_4.csv", y_pred_valid, delimiter=",",header='Listening_Time_minutes')

In [ ]:
lgb.plot_importance(model_4,max_num_features=20)

In [ ]:
#training's rmse: 5.5139	valid_1's rmse: 11.8236
X_train = pd.read_parquet('X_train_5.parquet.gzip') 
X_valid = pd.read_parquet('X_valid_5.parquet.gzip')
y_train = pd.read_csv('y_train_5.csv') 
y_valid = pd.read_csv('y_valid_5.csv')
X_test = pd.read_parquet('X_test_5.parquet.gzip')

y_train = y_train['Listening_Time_minutes']
y_valid = y_valid['Listening_Time_minutes']

X_train['Genre'] = X_train['Genre'].astype('category')
X_train['Podcast_Name'] = X_train['Podcast_Name'].astype('category')
X_train['Publication_Day'] = X_train['Publication_Day'].astype('category')
X_train['Publication_Time'] = X_train['Publication_Time'].astype('category')
X_train['Episode_Sentiment'] = X_train['Episode_Sentiment'].astype('category')

X_valid['Genre'] = X_valid['Genre'].astype('category')
X_valid['Podcast_Name'] = X_valid['Podcast_Name'].astype('category')
X_valid['Publication_Day'] = X_valid['Publication_Day'].astype('category')
X_valid['Publication_Time'] = X_valid['Publication_Time'].astype('category')
X_valid['Episode_Sentiment'] = X_valid['Episode_Sentiment'].astype('category')

X_test['Genre'] = X_test['Genre'].astype('category')
X_test['Podcast_Name'] = X_test['Podcast_Name'].astype('category')
X_test['Publication_Day'] = X_test['Publication_Day'].astype('category')
X_test['Publication_Time'] = X_test['Publication_Time'].astype('category')
X_test['Episode_Sentiment'] = X_test['Episode_Sentiment'].astype('category')

for x in X_train.columns[191:-5]:
    a = x + 'div' + 'Episode_Length_minutes'
    X_train[a] = X_train['Episode_Length_minutes'] / X_train[x]
    X_valid[a] = X_valid['Episode_Length_minutes'] / X_valid[x]
    X_test[a] = X_test['Episode_Length_minutes'] / X_test[x]

for x in [5,10,15,20,25,50,100]:
    name = 'Episode_Length_minutes' + str(x)
    X_train["Episode_Length_minutes_copy"] = X_train["Episode_Length_minutes"].copy()
    X_valid["Episode_Length_minutes_copy"] = X_valid["Episode_Length_minutes"].copy()
    X_test["Episode_Length_minutes_copy"] = X_test["Episode_Length_minutes"].copy()
    
    ser, bins = pd.qcut(X_train["Episode_Length_minutes_copy"], x, retbins=True, labels=False)
    X_train[name] = pd.cut(X_train['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
    X_valid[name] = pd.cut(X_valid['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')
    X_test[name] = pd.cut(X_test['Episode_Length_minutes_copy'], bins=bins, labels=False, include_lowest=True).astype('category')

for x in ['mean','quantile']:
    for w in ['Listening_Time_minutes']:
        for z in ['Episode_Length_minutes5','Episode_Length_minutes10','Episode_Length_minutes15','Episode_Length_minutes20'
                 ,'Episode_Length_minutes25','Episode_Length_minutes50','Episode_Length_minutes100']:
            name = w + x + z
            a = X_train.groupby(z)[w].apply(x).reset_index(name=name)
            X_train = X_train.merge(a,on=z,how='left')
            X_test = X_test.merge(a,on=z,how='left')      
            X_valid = X_valid.merge(a,on=z,how='left') 
print(X_valid.shape,len(y_valid))
print('8')

X_train['SinEpLen'] = np.sin(2*np.pi * X_train['Episode_Length_minutes']/60 )
X_train['CosEpLen'] = np.cos(2*np.pi * X_train['Episode_Length_minutes']/60 )
X_train["ELen_Int"] = np.floor(X_train["Episode_Length_minutes"])
X_train["ELen_Dec"] = X_train["Episode_Length_minutes"] - X_train["ELen_Int"]
X_train['Long_Episode'] = (X_train['Episode_Length_minutes'] > 75).astype(int)

X_test['SinEpLen'] = np.sin(2*np.pi * X_test['Episode_Length_minutes']/60 )
X_test['CosEpLen'] = np.cos(2*np.pi * X_test['Episode_Length_minutes']/60 )
X_test["ELen_Int"] = np.floor(X_test["Episode_Length_minutes"])
X_test["ELen_Dec"] = X_test["Episode_Length_minutes"] - X_test["ELen_Int"]
X_test['Long_Episode'] = (X_test['Episode_Length_minutes'] > 75).astype(int)

X_valid['SinEpLen'] = np.sin(2*np.pi * X_valid['Episode_Length_minutes']/60 )
X_valid['CosEpLen'] = np.cos(2*np.pi * X_valid['Episode_Length_minutes']/60 )
X_valid["ELen_Int"] = np.floor(X_valid["Episode_Length_minutes"])
X_valid["ELen_Dec"] = X_valid["Episode_Length_minutes"] - X_valid["ELen_Int"]
X_valid['Long_Episode'] = (X_valid['Episode_Length_minutes'] > 75).astype(int)

X_train = X_train.drop(columns=['Listening_Time_minutes'])
X_valid = X_valid.drop(columns=['Listening_Time_minutes'])

y_oof, y_pred_valid,model_5 = train(X_train,y_train,X_test,y_pred)
np.savetxt("y_oof_5.csv", y_oof, delimiter=",",header='Listening_Time_minutes')
np.savetxt("y_pred_valid_5.csv", y_pred_valid, delimiter=",",header='Listening_Time_minutes')
print('Done')

In [ ]:
df_subm = pd.read_csv('sample_submission.csv', index_col='id')
df_subm['Listening_Time_minutes'] = y_pred / 5
df_subm = df_subm.reset_index()
df_subm

In [ ]:
df_subm.to_csv('Kaggel_listentime_submission.csv',header=True,index=False)